In [ ]:
#|default_exp mic

# Streaming live audio from a microphone

Classes to stream audio from a microphone. Live audio is sent over a ZMQ port for further processing. 

First we import the needed python libraries. 

In [ ]:
#| export
import io
import sys 
import queue

import zmq
import fire
import numpy as np
import sounddevice as sd
import speech_recognition as sr
from pydub import AudioSegment
from fastcore.basics import store_attr

Next we import some default values from the `asr_teach.utils` module. 

In [ ]:
#| export 
from asr_teach.utils import SAMPLE_RATE, DEVICE, BLOCK_DURATION, DTYPE
from asr_teach.utils import ZMQ_ARGS, PROTOCOL, ADDR, PORT

## Base `AudioStream` class

The `AudioStream` class implements the basic, shared functionality to stream live audio from a mic. It takes the following arguments:  

- sample_rate: The sampling rate for the microphone audio.  
- device: The hardware ID of the microphone. Will often be `0` or `1` depending on the setup.  
- dtype: Data type for the recorded audio. Most microphone record  audio as Short Ints (16 bits aka `np.int16`).  
- protocol: The transport protocol for the ZMQ port (default: TCP).  
- addr: The address (typically an IP address) for the ZMQ connection.  
- port: The specific port that audio will be sent over.  

The class creates a ZMQ socket to carry the output audio samples.

In [ ]:
#| export

class AudioStream:
    def __init__(self,
                 sample_rate: float = None,
                 device: int = 0,
                 dtype: str = DTYPE,
                 addr: str = ADDR,
                 port: int = PORT,
                 protocol: str = PROTOCOL):
        """Base audio stream class.
        """
        store_attr()
        
        # creates the ZMQ socket
        context = zmq.Context()
        socket = context.socket(zmq.PUSH)
        socket.bind(f'{protocol}://{addr}:{port}')
        self.socket = socket

        
    def run(self):
        """Runs the microphone streaming.
        """
        raise NotImplementedError
        
    def setup_stream(self):
        """Initializes and sets up a specific stream.
        """
        raise NotImplementedError
        
    def send_audio(self, indata, flags=0, copy=True, track=False):
        '''Sends a numpy array as a multi-part message.
    
        The first part of the message has the shape and type of the array.
        The second message has the raw array bytes.
        Both pieces of information are needed to correctly deserialize the array at the receiver. 
        '''
        # sends the array's shape and type info
        md = {'dtype': str(indata.dtype),
              'shape': indata.shape}
        self.socket.send_json(md, zmq.SNDMORE)
        # sends the raw array bytes
        self.socket.send(indata, flags, copy=copy, track=track)
        

### Streaming via the `sounddevice` library.

The `SoundDeviceMic` class uses the `sounddevice` python library to stream continuous audio from a microphone. 

The `block_duration`, given in milliseconds, determines the size of the recorded audio buffer. For example if `block_duration = 2500` then 2500 milliseconds (aka 2.5 seconds) of audio will be buffered in and made ready for processing.    

The `sounddevice` library records audio in a separate thread, and we can process these buffered samples from our main thread via a `callback` function. In our case, the callback function places the audio samples on a queue. We then pop samples from the queue and send them over the ZMQ socket.  

We use a queue as a middleman because the `sounddevice` library makes a few strong assumptions about the callback. In summary, the callback needs to be as fast as possible so that it does not block the audio stream in the separate thread. This means we should avoid IO and network calls from the callback itself. For this reason our callback simply places audio samples on the queue and allows the separate thread to continue at will. 

In [ ]:
#| export

class SoundDeviceMic(AudioStream):
    def __init__(self, *args,
                 block_duration: int = 2500,
                 **kwargs):
        '''Streams continuous live audio with the `sounddevice` library.
        '''
        super().__init__(*args, **kwargs)
        store_attr()
        self.setup_stream()
        
    def setup_stream(self):
        '''Prepares the `sounddevice` stream.
        
        First, we figure out the sampling rate and audio buffer sizes.
        Next we initialize a queue for the buffered audio samples.
        
        Lastly, we create the microphone stream with the callback that processes the recorded audio.
        '''
        
        # setting the sample rate and mic buffer size
        self.sample_rate = self.sample_rate or sd.query_devices(self.device, 'input')['default_samplerate']
        self.num_samples = int(self.sample_rate * self.block_duration)
        self.blocksize = self.num_samples // 1000
        
        # queue to hold the mic data
        self.q = queue.Queue()

        def enqueue_audio(indata, frames, time, status):
            '''Places buffered audio data on the queue.
            '''
            if any(indata):
                self.q.put(indata)
        
        # create the microphone streaming object
        self.stream = sd.InputStream(device=self.device,
                                     channels=1,
                                     callback=enqueue_audio,
                                     dtype=self.dtype,
                                     blocksize=self.blocksize,
                                     samplerate=self.sample_rate)
        
        
    def run(self):
        '''Streams audio until the user stops or interrupts the process.
        
        We continuously check whether there are any audio samples on the queue.
        If there are, we send them over the ZMQ socket.  
        '''
        # start the microphone stream
        self.stream.start()
        print('Streaming live mic audio...')
        
        while True:
            # send any audio on the queue
            try:
                if not self.q.empty():
                    self.send_audio(self.q.get())
                    
            except KeyboardInterrupt:
                print('User keyboard interrupt, stopping mic...')
                self.stream.close()
                break
                
            except Exception as e:
                print(f'Exception: {e}')
                raise
                
        print('Live mic stopped.')
            

## Streaming via the `SpeechRecognition` library

The `SpeechRecogMic` is a more advanced audio stream using the `SpeechRecognition` library.  

This stream takes a few arguments to run energy detection on the microphone. Instead of continuously streaming potentially empty audio, this class only records and sends audio when the microphone activity crosses a certain energy threshold.  

The stream will wait and hold until the energy threshold is crossed.  

Then, it waits for the user to stop speaking before stopping the recording and sending the audio over. The wait time for a recording to be over is set via the `pause` argument.  

In [ ]:
#| export

class SpeechRecogMic(AudioStream):
    def __init__(self, *args,
                 energy: int = 500,
                 pause: float = 0.8,
                 dynamic_energy: bool = False,
                 **kwargs):
        '''Streams microphone audio with the SoundRecognizer library.
        '''
        super().__init__(*args, **kwargs)
        store_attr()
        self.setup_stream()
        
    def setup_stream(self):
        '''Creates the energy-aware speech recognizer and microphone stream.
        '''
        
        # loads the speech recognizer with energy and pause parameters
        recog = sr.Recognizer()
        recog.energy_threshold = self.energy
        recog.pause_threshold = self.pause
        recog.dynamic_energy_threshold = self.dynamic_energy
        self.recog = recog
        
        # initializes the chosen microphone stream
        self.mic = sr.Microphone(device_index=self.device,
                                 sample_rate=self.sample_rate)

    def run(self):
        '''Streams audio until the user stops or interrupts the process.
        
        The recognizer `recog` starts recording once the microphone's energy  
        crosses the threshold. Then, it keeps recording until the energy drops
        for `pause` seconds.  
        
        The raw buffer bytes are then cast as a numpy short-int array and sent
        over the ZMQ port.
        '''
        with self.mic as source:
            print("Starting mic stream...")
            
            while True:
                # record audio stream into wav
                try:
                    data = self.recog.listen(source)
                    audio = np.frombuffer(data.frame_data, dtype=np.int16)
                    #audio = audio.flatten().astype(np.float32) / 32768.0
                    self.send_audio(audio)
                    
                except KeyboardInterrupt:
                    print('User keyboard interrupt, stopping mic...')
                    break

                except Exception as e:
                    print(f'Exception: {e}')
                    raise
                    
            print('Mic stopped.')